**Assignment**

In [1]:
import requests
import json

url = "https://storage.googleapis.com/xcc-de-assessment/events.json"
response = requests.get(url)

# Split the content by lines
lines = response.text.splitlines()

# Parse each line as an individual JSON object
data = [json.loads(line) for line in lines]



In [2]:
#checking the number of rows
total_rows = len(data)
print(total_rows)


667961


In [3]:
#filtering the customers with null id 

logged_in_data = [event for event in data if event["event"]["customer-id"] is not None]

total_rows = len(logged_in_data)
print(total_rows)

191011


==> from a total of 667961 rows we have 191011 rows with non-null customer_id

In [4]:
# I'm gonna assume based in the assignment reauirement that the time between each session is 30 minutes of inactivity 

import datetime
logged_in_data.sort(key=lambda x: (x["event"]["customer-id"], x["event"]["timestamp"]))

sessions = []
session = []
prev_timestamp = None

for event in logged_in_data:
    current_timestamp = datetime.datetime.fromisoformat(event["event"]["timestamp"])
    if prev_timestamp and (current_timestamp - prev_timestamp).seconds > 1800:  # 30 minutes in seconds
        sessions.append(session)
        session = []
    session.append(event)
    prev_timestamp = current_timestamp

if session:
    sessions.append(session)


In [5]:
pip install psycopg2


Note: you may need to restart the kernel to use updated packages.


In [6]:

import psycopg2

# Connection details for PostgreSQL database on Docker
DB_NAME = "xcceleratedpostgresdb"
DB_USER = "postgres"  # user
DB_PASSWORD = "AzerKimo2022@@@"
DB_HOST = "localhost"  #  connecting from the host machine
DB_PORT = "5432"  # default port for PostgreSQL

# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cursor = conn.cursor()





In [7]:
#i am using a PostgreSQL database called xcceleratedpostgresdb running on a docker container  

import psycopg2

# Connection details to PostgreSQL
DB_NAME = "xcceleratedpostgresdb"
DB_USER = "postgres"
DB_PASSWORD = "AzerKimo2022@@@"
DB_HOST = "localhost"
DB_PORT = "5432"

# Try connecting to the database
try:
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )

    # Create a cursor object to interact with PostgreSQL
    cursor = conn.cursor()

    # Execute a simple query to fetch PostgreSQL version
    cursor.execute("SELECT version();")
    version = cursor.fetchone()

    # Print the version
    print("Connected to:", version[0])

except psycopg2.OperationalError as e:
    print(f"Failed to connect to the database! Error: {e}")
    

    # Ensure to close the connection and cursor if they exist
    #if cursor:
    #   cursor.close()
    # if conn:
    #    conn.close() '''


Connected to: PostgreSQL 16.0 (Debian 16.0-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit


In [8]:
# Create a table for sessions
cursor.execute("""
CREATE TABLE IF NOT EXISTS sessions (
    session_id SERIAL PRIMARY KEY,
    customer_id INTEGER,
    start_time TIMESTAMP,
    end_time TIMESTAMP,
    duration INTEGER
)
""")

# Insert sessions into the table
for idx, session in enumerate(sessions):
    start_time_str = session[0]["event"]["timestamp"]
    end_time_str = session[-1]["event"]["timestamp"]
    
    start_time = datetime.datetime.fromisoformat(start_time_str)
    end_time = datetime.datetime.fromisoformat(end_time_str)
    
    duration = (end_time - start_time).seconds
    customer_id = session[0]["event"]["customer-id"]

    cursor.execute("INSERT INTO sessions (customer_id, start_time, end_time, duration) VALUES (%s, %s, %s, %s)", 
                   (customer_id, start_time_str, end_time_str, duration))
conn.commit() 

**Step 2: Publish the Results**

1. Design the API
2. Implementation

In [9]:
# Set up Flask and threading since i am working in ipynb file 

from flask import Flask, jsonify
import threading

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello, Xccelerated!"

def run():
    app.run(debug=True, use_reloader=False)  # Set use_reloader to False to avoid the loop restart

t = threading.Thread(target=run)
t.start()




 * Serving Flask app '__main__'
 * Debug mode: on


In [10]:
# Database Connection
import psycopg2

def get_db_connection():
    # Define your connection parameters
    dbname = "xcceleratedpostgresdb"
    user = "postgres"
    password = "AzerKimo2022@@@"
    host = "localhost"
    port = "5432"
    
    # Establish and return the connection
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    return conn


In [11]:
from flask import Flask, jsonify
from flask_sqlalchemy import SQLAlchemy
import psycopg2

app = Flask(__name__)

@app.route('/metrics/orders', methods=['GET'])
def get_order_metrics():
    # Connection details to PostgreSQL
    DB_NAME = "xcceleratedpostgresdb"
    DB_USER = "postgres"
    DB_PASSWORD = "AzerKimo2022@@@"
    DB_HOST = "localhost"
    DB_PORT = "5432"

    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        cursor = conn.cursor()
        
        # SQL query to calculate median visits before order
        median_visits_query = """
        SELECT
            percentile_cont(0.5) WITHIN GROUP (ORDER BY visits) AS median_visits
        FROM (
            SELECT
                customer_id,
                COUNT(DISTINCT session_id) AS visits
            FROM
                sessions
            WHERE
                customer_id IS NOT NULL
            GROUP BY
                customer_id
        ) subquery
        """
        
        # SQL query to calculate median session duration before order
        median_duration_query = """
        SELECT
            percentile_cont(0.5) WITHIN GROUP (ORDER BY duration) AS median_duration_minutes
        FROM (
            SELECT
                customer_id,
                MIN(duration) AS duration
            FROM
                sessions
            WHERE
                customer_id IS NOT NULL
            GROUP BY
                customer_id
        ) subquery
        """
        
        cursor.execute(median_visits_query)
        median_visits = cursor.fetchone()[0]
        
        cursor.execute(median_duration_query)
        median_duration_minutes = cursor.fetchone()[0]
        
        # Return the metrics as JSON response
        metrics = {
            "median_visits_before_order": median_visits,
            "median_session_duration_minutes_before_order": median_duration_minutes
        }
        
    except psycopg2.Error as e:
        # Handle database connection error
        metrics = {"error": str(e)}
        
    finally:
        # Close the cursor and connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()
    
    return jsonify(metrics)

if __name__ == "__main__":
    app.run(debug=True)

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\mamorpc\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import datetime
import psycopg2
from flask import Flask, jsonify
from flask_sqlalchemy import SQLAlchemy

# Database configuration
DB_NAME = "xcceleratedpostgresdb"
DB_USER = "postgres"
DB_PASSWORD = "AzerKimo2022@@@"  # Consider using environment variables for sensitive info
DB_HOST = "localhost"
DB_PORT = "5432"

# Create Flask app
app = Flask(__name__)

# SQLAlchemy configuration for PostgreSQL
app.config['SQLALCHEMY_DATABASE_URI'] = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Initialize SQLAlchemy for the app
db = SQLAlchemy(app)


# Model for the 'sessions' table
class Session(db.Model):
    __tablename__ = 'sessions'
    __table_args__ = {'schema': 'public'}
    session_id = db.Column(db.Integer, primary_key=True)
    customer_id = db.Column(db.Integer)
    start_time = db.Column(db.DateTime)
    end_time = db.Column(db.DateTime)
    duration = db.Column(db.Integer)

    def as_dict(self):
        return {c.name: getattr(self, c.name) for c in self.__table__.columns}


def get_db_connection():
    """
    Establish and return a connection to the PostgreSQL database.
    """
    return psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )


@app.route('/metrics/orders', methods=['GET'])
def get_order_metrics():
    """
    Return metrics on median visits and session duration before an order.
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        # SQL queries to calculate metrics
        median_visits_query = """
        SELECT
            percentile_cont(0.5) WITHIN GROUP (ORDER BY visits) AS median_visits
        FROM (
            SELECT
                customer_id,
                COUNT(DISTINCT session_id) AS visits
            FROM
                sessions
            WHERE
                customer_id IS NOT NULL
            GROUP BY
                customer_id
        ) subquery
        """
        
        median_duration_query = """
        SELECT
            percentile_cont(0.5) WITHIN GROUP (ORDER BY duration) AS median_duration_minutes
        FROM (
            SELECT
                customer_id,
                MIN(duration) AS duration
            FROM
                sessions
            WHERE
                customer_id IS NOT NULL
            GROUP BY
                customer_id
        ) subquery
        """

        cursor.execute(median_visits_query)
        median_visits = cursor.fetchone()[0]

        cursor.execute(median_duration_query)
        median_duration_minutes = cursor.fetchone()[0]

        metrics = {
            "median_visits_before_order": median_visits,
            "median_session_duration_minutes_before_order": median_duration_minutes
        }

    except psycopg2.Error as e:
        metrics = {"error": str(e)}

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

    return jsonify(metrics)


if __name__ == "__main__":
    app.run(debug=True)
